In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.special import gammainc

# Functions 

In [3]:
def sigma(sig_ln, F0, alpha):
    ''' Compute sigma'''
    return sig_ln * F0**(1-alpha)

def X(F, alpha, sig):
    ''' X transform variable '''
    return F**(2 * (1-alpha)) / (sig**2 * (1-alpha)**2)

def F_Xinv(X, alpha, sig):
    ''' F from X'''
    return (X * (sig**2 * (1-alpha)**2)) ** (1/(2*(1-alpha)))

def delta(alpha):
    ''' delta from alpha'''
    return (1 - 2*alpha) / (1 - alpha)

def em_CEV(T, dt, F0, alpha, sig):
    ''' Euler scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X0 = X(F0, alpha, sig)
    d = delta(alpha)
    X_emC, X_em = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # euler (X + XdX)=(X + X * (d*dt + 2X^(1/2)dW))
        dX = d*dt + 2*np.sqrt(X_emC)*dW[j]
        X_emC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_emC <= 0:
            # fill with 0's since it is absorbed
            X_em += [0 for i in range(N - len(X_em))]
            hit_zero = True
            break
        X_em.append(X_emC)
    
    return X_em, hit_zero

def em_CEV(T, dt, F0, sig):
    ''' Euler scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X_emC, X_em = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # euler (X + XdX)=(X + X * (d*dt + 2X^(1/2)dW))
        dX = d*dt + 2*np.sqrt(X_emC)*dW[j]
        X_emC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_emC <= 0:
            # fill with 0's since it is absorbed
            X_em += [0 for i in range(N - len(X_em))]
            hit_zero = True
            break
        X_em.append(X_emC)
    
    return X_em, hit_zero


def mil_CEV(T, dt, F0, alpha, sig):
    ''' Milstein scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X0 = X(F0, alpha, sig)
    d = delta(alpha)
    X_milC, X_mil = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # Milstein 
        dX = d*dt + 2*np.sqrt(X_milC)*dW[j] + 2*(dW[j]**2 - dt)
        X_milC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_milC <= 0:
            # fill with 0's since it is absorbed
            X_mil += [0 for i in range(N - len(X_mil))]
            hit_zero = True
            break
        X_mil.append(X_milC)
    
    return X_mil, hit_zero
    
    

# CEV Process for K = 90, 100, 110

In [132]:

# parameters
dt = 2**(-4)
F = 100
alpha = 0
sig_ln = 0.5
T = 4
K = 100
# transform sig to correct dimension
sig = sigma(sig_ln, F, alpha)

# Monte Carlo of CEV Process
N = 10**(4)
t = np.arange(dt, T + dt, dt)
X_T = np.zeros(N)
num_absorbed = 0
for i in range(N):
    em, hz = mil_CEV(T, dt, F, alpha, sig)
    num_absorbed += int(hz)
    # save final X values
    X_T[i] = em[-1]
    # plt.plot(t, em, alpha=0.5)

# compute inverse to get F
F_T = F_Xinv(X_T, alpha, sig)
# compute C by subtracting K and taking max. Anywhere less than K is 0, otherwise F - K
C = np.where(F_T<K, 0, F_T-K)
P = C - (F_T - K)
# print(C)
# print(C[np.argwhere(C).flatten()])
print(f"E[C] = {C.mean()}, STD[C] = {C.std()/(N**(1/2))}")
print(f"E[P] = {P.mean()}, STD[P] = {P.std()/(N**(1/2))}")
print(f"Simulated Absorbed Ratio = {num_absorbed/N}")
print(f"Analytical Absorbed Ratio = {1-gammainc(-(delta(alpha)/2 - 1), X(F, alpha, sig)/(2*T))}")
# plt.ylim(0, 10)
# plt.show()

E[C] = 39.20364786952627, STD[C] = 0.6985665361447374
E[P] = 38.69580214169203, STD[P] = 0.41111455517833256
Simulated Absorbed Ratio = 0.1848
Analytical Absorbed Ratio = 0.19357461811489696


In [84]:
def analytical_C(F,K,T, alpha, sig):
    K_tilde = X(K,alpha, sig)
    X0 = X(F,alpha, sig)
    v = delta(alpha)/2 - 1
    C = F * (1 - stats.ncx2.cdf(x = K_tilde/T, df = 4 - delta(alpha), nc = X0/T)) - K * stats.ncx2.cdf(x = X0/T, df = 2 - delta(alpha),nc = K_tilde/T)
    
    return C
    # C = np.where(C < 0, 0, C)
    # C_idx = np.argwhere(C).flatten()
    # print(C[C_idx].mean())
    # print(C.mean())


In [88]:
F = 100
alpha = -2
sig_ln = 0.5
T = 4
K = 100
# transform sig to correct dimension
sig = sigma(sig_ln, F, alpha)
print(F, alpha, sig_ln, sig, T, K)
ana_C = analytical_C(F, K, T, alpha, sig)
ana_P = ana_C + K - F
print(ana_C, ana_P)

100 -2 0.5 500000.0 4 100
34.42927514295155 34.42927514295155


In [24]:
F = 100
alpha = 0
sig_ln = 0.5
T = 4
K = 100
# transform sig to correct dimension
sig = sigma(sig_ln, F, alpha)

v = delta(alpha)/2 - 1
X0 = X(F, alpha, sig)


N = 10**4
U = np.random.uniform(size=N)
Umax = gammainc(-v, X0/(2*T))
invalid = U > Umax
XT = np.ones(N)
XT[invalid] = 0
valid = ~invalid

XT[valid] = 1-stats.ncx2.isf(X0/T, df = 2 - delta(alpha), nc = Umax - U[valid])

print(XT)

[1. 1. 1. ... 1. 1. 1.]


In [123]:
dt = 2**(-4)

# parameters
F = 100
sig_ln = 0.2
T = 1
K = 100
alpha = 7

sig = sigma(sig_ln, F, alpha)

# Monte Carlo
N = 1000000
t = np.arange(dt, T + dt, dt)
X_T = []
counter = 0
for i in range(N):
    em, hit_zero = em_CEV(T, dt, F, alpha, sig)
    if hit_zero:
        continue
    X_T.append(em[-1])
    counter += 1
    # plt.plot(t, em, alpha=0.5)
    
X_T = np.array(X_T)
# compute inverse to get F
F_T = F_Xinv(X_T, alpha, sig)
# compute C by subtracting K and taking max. Anywhere less than K is 0, otherwise F - K
C = np.where(F_T<K, 0, F_T-K)
P = C - (F_T - K)
# print(C)
# print(C[np.argwhere(C).flatten()])
print(f"E[C] = {C.mean()}, STD[C] = {C.std()/(counter**(1/2))}")
print(f"E[P] = {P.mean()}, STD[P] = {P.std()/(counter**(1/2))}")
# print(f"Simulated Absorbed Ratio = {num_absorbed/N}")
# print(f"Analytical Absorbed Ratio = {1-gammainc(-(delta(alpha)/2 - 1), X(F, alpha, sig)/(2*T))}")
# plt.ylim(0, 10)
# plt.show()

# plt.ylim(0, 10)
# plt.show()

KeyboardInterrupt: 

In [111]:
# Local martingale for F_T

def analytical_C_2(F, K,T, alpha, sig):
    K_tilde = X(K,alpha, sig)
    X0 = X(F,alpha, sig)
    print(delta(alpha))
    v = delta(alpha)/2 - 1
    print(v)
    F_T = F * (gammainc(v, X0/(2*T)))

    X_T = X(F_T, alpha, sig)
    print(F_T, X_T, X0)

    C = F * (gammainc(v, X0/(2*T)) - stats.ncx2.cdf(x = X0/T, df = delta(alpha) - 2, nc = X0/T)) - K * stats.ncx2.cdf(x = K_tilde/T, df = delta(alpha),nc = X0/T)

    print(C)
    # C = np.where(C < 0, 0, C)
    # C_idx = np.argwhere(C).flatten()
    # print(C[C_idx].mean())
    # print(C.mean())



In [125]:
F = 100
sig_ln = 0.2
T = 1
K = 100
alpha = 6.5
sig = sigma(sig_ln, F, alpha)
analytical_C_2(F, K, T, alpha, sig)

2.1818181818181817
0.09090909090909083
93.62078925858074 1.7065513114087554 0.8264462809917357
2.0604360207136097


In [63]:
delta(10)

2.111111111111111